In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw03.ipynb")

# Homework 03: Food Safety (Continued)

You must submit this assignment to Gradescope by the on-time deadline. **We strongly encourage you to plan to submit your work to Gradescope several days (hours) before the stated deadline.** This way, you will have ample time to reach out to staff for support if you encounter difficulties with submission. While course staff is happy to help guide you with submitting your assignment ahead of the deadline, we will not respond to last-minute requests for assistance (TAs need to sleep, after all!).

Please read the instructions carefully when you are submitting your work to Gradescope.




## This Assignment

In this homework, we will continue our exploration of restaurant food safety scores for restaurants in San Francisco. The main goal for this assignment is to focus more on the analysis of the dataset, building on the data cleaning we have done earlier in HW 02. 


After this homework, you should be comfortable with:
* Reading `pandas` documentation and using `pandas` methods
* Using `.groupby` with different aggregation functions
* Chaining different `pandas` functions and methods to find answers to exploratory questions



## Before You Start

For each question in the assignment, please write down your answer in the answer cell(s) right below the question. 

We understand that it is helpful to have extra cells breaking down the process towards reaching your final answer. If you happen to create new cells below your answer to run code, **NEVER** add cells between a question cell and the answer cell below it. It will cause errors when we run the autograder, and it will sometimes cause a failure to generate the PDF file.

**Important note: The local autograder tests will not be comprehensive. You can pass the automated tests in your notebook but still fail tests in the autograder.** Please be sure to check your results carefully.

Finally, unless we state otherwise, **do not use for loops or list comprehensions**. The majority of this assignment can be done using built-in commands in `pandas` and `NumPy`.  Our autograder isn't smart enough to check, but you're depriving yourself of key learning objectives if you write loops / comprehensions, and you also won't be ready for the midterm.

### Debugging Guide
If you run into any technical issues, we highly recommend checking out the [Debugging Guide](https://mtu.instructure.com/courses/1527249/pages/debugging-guide). In this guide, you can find general questions about Jupyter notebooks / Datahub, Gradescope, and common pandas errors.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.style.use('fivethirtyeight')

<br/><br/>
<hr style="border: 5px solid #8a8c8c;" />
<hr style="border: 1px solid #ffcd00;" />

In HW 2, we took you through the entire process of reading data from a file to perform some exploration of the data. Here, we again load the dataset that we will be using in HW 3 along with some of the columns we had added in HW 2. For any additional context regarding the dataset, feel free to revisit HW 2.

In [ ]:
bus = pd.read_csv('data/bus.csv', encoding='ISO-8859-1').rename(columns={"business id column": "bid"})
bus['postal5'] = bus['postal_code'].str[:5]
ins = pd.read_csv('data/ins.csv')
ins['timestamp'] = pd.to_datetime(ins['date'], format='%m/%d/%Y %I:%M:%S %p')
ins['bid'] = ins['iid'].str.split("_", expand=True)[0].astype(int) 

# This code is essential for the autograder to function properly. Do not edit.
ins_test = ins

In [ ]:
bus.head()

# Part 1: Zip-code Analysis 

Let's try and figure out whether there are any differences between names of restaurants located in even and odd ZIP codes (specifically using the 5-digit postal codes). We will break down this analysis into steps with the end goal of figuring out the restaurant with the shortest name with an even ZIP code.

<br>

--- 

## Question 1.1 

First, calculate the length of the `name` of each of the restaurants in `bus` and store the values in the DataSeries `name_length`.  **Do not use for loops or list comprehensions.**

In [ ]:
name_length = ...
name_length

In [ ]:
grader.check("q11")

<br>

---

## Question 1.2 

To work the 5-digit ZIP codes and check whether they are even or odd, we need to ensure that there are no None values contained. Create a new `DataFrame` `bus_valid` which is a [`copy` of `bus`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.copy.html#pandas.DataFrame.copy) and only contains rows with `postal5` values that are not None. You may find the `.isna()` function useful! For the rest of this question, we will be working with `bus_valid`.

We will fist check for valid ZIP codes like was formulated in HW 02 Question 1.3. 

In [ ]:
valid_zips = pd.read_json('data/sf_zipcodes.json')['zip_codes']
valid_zips = valid_zips.astype("string")

bus.loc[~bus['postal5'].isin(valid_zips), 'postal5'] = None

In [ ]:
bus_valid = ...
bus_valid.head()

In [ ]:
grader.check("q12")

<br>

---

## Question 1.3

Now, assign `is_even` to a boolean **Series** that indicates whether the corresponding 5-digit ZIP code in `bus_valid` is even or odd. Remember to keep in mind the data type of `postal5`!

Hint: You might find the mod operation `%` useful here!

In [ ]:
is_even = ...

In [ ]:
grader.check("q13")

<br>

---

## Question 1.4 

Using the Series you created above, store the name of the business with the shortest name amongst all businesses located in even ZIP codes in `shortest_name_even`. Make sure that `shortest_name_even` contains a **string** with your answer. 

Note: When sorting, please break ties with reverse alphabetical order. Feel free to reference the [sort_values documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) to see how you can sort by multiple values.

In [ ]:
...
shortest_name_even

In [ ]:
grader.check("q14")

<br/><br/>
<hr style="border: 5px solid #8a8c8c;" />
<hr style="border: 1px solid #ffcd00;" />

# Part 2: Inspecting the Inspections 

<br/>

---


## Question 2.1

Let's start by looking again at the first 5 rows of `ins` to see what we're working with.

In [ ]:
ins.head(5)

To better understand how the scores have been allocated, examine how the maximum score varies for each type of inspection. Create a `DataFrame` object `ins_score_by_type`, indexed by all the inspection types (e.g., New Construction, Routine - Unscheduled, etc.), with a single column named `max_score` containing the highest score received. You may find `df.rename()` to be useful!

In [ ]:
ins_score_by_type = ...
ins_score_by_type

In [ ]:
grader.check("q21")

<br/>

---

## Question 2.2


Given the variability of `ins['score']` observed in 1a, let's examine the inspection scores `ins['score']` further.

In [ ]:
ins['score'].value_counts().head()

There are a large number of inspections with a `score` of `-1`. These are probably missing values. Let's see what types of inspections have scores and which do not (score of -1).  We have defined for you a new column `'Missing Score'` that shows `True` if the score for that business is `-1` to help you out with the analysis. 

Use `.groupby` to find out the number of scores that every combination of `type` and `Missing Score` can take on. The result should be a **`DataFrame`** that should look **exactly** as shown below:

<center> <img src="pics/1b.png" width="400"/> 


In [ ]:
ins['Missing Score'] = (ins['score'] == -1).astype("str") 
ins_missing_score_group = ...
ins_missing_score_group

In [ ]:
grader.check("q22")

<!-- BEGIN QUESTION -->

<br/>

---

## Question 2.3


Using `.groupby` to perform the above analysis gave us a `DataFrame` that wasn't the most readable at first glance. There are better ways to represent the above information that take advantage of the fact that we are looking at combinations of two variables. It's time to pivot (pun intended)!

Create the following `DataFrame`, and assign it to to the variable `ins_missing_score_pivot`. You'll want to use the `pivot_table` method of the `DataFrame` class, which you can read about in the `pivot_table` [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html). Once you create `ins_missing_score_pivot`, add another column titled `'Total'`, which contains the total number of inspections of that `type`. Sort the table by descending order of `'Total'`.

**Hint:** Consider what happens if there are no values that correspond to a particular combination of `'Missing Score'` and `'type'`. Looking at the documentation for `pivot_table`, is there any function argument that allows you to specify what value to fill in?

**Hint:** You may want to look at the documentation for [`droplevels` documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.droplevel.html#pandas.DataFrame.droplevel)

If you've done everything right, you should observe that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections and that `ins_missing_score_pivot` looks exactly like below:


<center><img src="pics/1c.png"  width="400">


In [ ]:
ins['Missing Score'] = (ins['score'] == -1).astype("str")
ins_missing_score_pivot = ...

...

ins_missing_score_pivot 

In [ ]:
grader.check("q23")

<!-- END QUESTION -->

Notice that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections. It is reasonable for inspection types such as `New Ownership` and `Complaint` to have no associated inspection scores, but we might be curious why there are no inspection scores for the `Reinspection/Followup` inspection type.

<br/><br/>
<hr style="border: 5px solid #8a8c8c;" />
<hr style="border: 1px solid #ffcd00;" />

# Part 3: Joining Data Across Tables

In this question, we will start to connect data across multiple tables. We will be using the `merge` function. 

<br/>

--- 

## Question 3.1

Let's figure out which restaurants had the lowest scores. Before we proceed, let's filter out missing scores from `ins` so that negative scores don't influence our results. 

Note that there might be something interesting we could learn from businesses with missing scores, but we are omitting such analysis from this homework.  

Note: We have no idea if there is actually anything interesting to learn as we have not attempted this ourselves.

In [ ]:
ins = ins[ins["score"] > 0]

We'll start by creating a new `DataFrame` called `ins_named`. It should be exactly the same as `ins`, except that it should have the name and address of every business, as determined by the `bus` `DataFrame`. 

**Hint**: Use the `merge` method to join the `ins` `DataFrame` with the appropriate portion of the `bus` `DataFrame`. See the official [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) on how to use `merge`. The first few rows of the resulting `DataFrame` you create are shown below:

<img src="pics/2a.png" width="1080"/>

In [ ]:
...
ins_named.head()

In [ ]:
grader.check("q31")

<!-- BEGIN QUESTION -->

<br/>

--- 

## Question 3.2

Let's look at the 20 businesses in `ins_named` with the lowest **median** score. Order your results by the median score followed by the business name to break ties. The resulting table should look like the table below.

This one is pretty challenging! Don't forget to rename the `score` column. 

**Hint**: The `agg` function can accept a dictionary as an input. See the [agg documentation](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.agg.html#pandas.core.groupby.DataFrameGroupBy.agg). Additionally, when thinking about what aggregation functions to use, ask yourself what value would be in the `"name"` column for each entry across the group? Can we select just one of these values to represent the whole group?

As usual, **YOU SHOULD NOT USE LOOPS OR LIST COMPREHENSIONS**. Try and break down the problem piece by piece instead, gradually chaining together different `pandas` functions. Feel free to use more than one line!

<table border="1" class="dataframe">  <thead>    
    <tr style="text-align: right;">      <th></th>      <th>name</th>      <th>median score</th>    </tr> 
    <tr  align="right">  <th align="right">bid</th>      <th></th>      <th></th>    </tr> </thead>  <tbody>    
    <tr  align="right">      <th>84590</th>      <td>Chaat Corner</td>      <td>54.0</td>    </tr>    
    <tr  align="right">        <th>90622</th>      <td>Taqueria Lolita</td>      <td>57.0</td>    </tr>    
    <tr  align="right">         <th>94351</th>      <td>VBowls LLC</td>      <td>58.0</td>    </tr>    
    <tr  align="right">          <th>69282</th>      <td>New Jumbo Seafood Restaurant</td>      <td>60.5</td>    </tr>    
    <tr  align="right">         <th>1154</th>      <td>SUNFLOWER RESTAURANT</td>      <td>63.5</td>    </tr>  
    <tr  align="right">          <th>93150</th>      <td>Chez Beesen</td>      <td>64.0</td>    </tr>   
    <tr  align="right">     <th>39776</th>      <td>Duc Loi Supermarket</td>      <td>64.0</td>    </tr>  
    <tr  align="right">         <th>78328</th>      <td>Golden Wok</td>      <td>64.0</td>    </tr>  
    <tr  align="right">          <th>69397</th>      <td>Minna SF Group LLC</td>      <td>64.0</td>    </tr>     
    <tr  align="right">        <th>93502</th>      <td>Smoky Man</td>      <td>64.0</td>    </tr>    
    <tr  align="right">           <th>98995</th>      <td>Vallarta's Taco Bar</td>      <td>64.0</td>    </tr>    
    <tr  align="right">         <th>10877</th>      <td>CHINA FIRST INC.</td>      <td>64.5</td>    </tr>    
    <tr  align="right">        <th>71310</th>      <td>Golden King Vietnamese Restaurant</td>      <td>64.5</td>    </tr>     
    <tr  align="right">          <th>89070</th>      <td>Lafayette Coffee Shop</td>      <td>64.5</td>    </tr>
    <tr  align="right">          <th>71008</th>      <td>House of Pancakes</td>      <td>65.0</td>    </tr> 
    <tr  align="right">         <th>2542</th>      <td>PETER D'S RESTAURANT</td>      <td>65.0</td>    </tr>           
    <tr  align="right">        <th>3862</th>      <td>IMPERIAL GARDEN SEAFOOD RESTAURANT</td>      <td>66.0</td>    </tr>   
    <tr  align="right">         <th>61427</th>      <td>Nick's Foods</td>      <td>66.0</td>    </tr>    
    <tr  align="right">          <th>72176</th>      <td>Wolfes Lunch</td>      <td>66.0</td>    </tr>    
    <tr  align="right">        <th>89141</th>      <td>Cha Cha Cha on Mission</td>      <td>66.5</td>    </tr>  </tbody></table>

In [ ]:
twenty_lowest_scoring = ... 

# DO NOT USE LIST COMPREHENSIONS OR LOOPS OF ANY KIND!!!

...

twenty_lowest_scoring

In [ ]:
grader.check("q32")

<!-- END QUESTION -->

<br/>

--- 
## Question 3.3

Let's figure out which restaurant had the worst score ever (single lowest score). 

In the cell below, assign `worst_restaurant` to the name of the restaurant with the **lowest inspection score ever**. We should not be considering restaurants with missing scores, so this should not be a retaurant that has a score of `-1`. For fun: Look up the reviews for this restaurant on Yelp. Do you see any reviews that indicate this restaurant had health inspection issues?


In [ ]:
worst_restaurant = ...

worst_restaurant

In [ ]:
grader.check("q33")

<!-- BEGIN QUESTION -->

<br/>

--- 
## Question 3.4

Did this restaurant clean up its act? Look in the database to see if it scored better on its next inspection. Write code to assign `cleaned_up` to `True` or `False`, depending on whether it performed better or not.

In [ ]:
# SCRATCH WORK

In [ ]:
cleaned_up = ...

In [ ]:
grader.check("q34")

<!-- END QUESTION -->

## Summary of Inspections Data

We have done a lot in this homework! 
 
- Broke down the inspection scores in detail using `.groupby` and `pivot_table`
- Joined the business and inspection data and identified the name of the restaurant with the worst rating


Over the course of the last 2 homeworks, we hope you have become more familiar with `pandas` - in terms of identifying when to use particular functions, how they work, when they can support EDA - as well as with EDA and Data Cleaning, as part of the broader Data Science Lifecycle. These tools will serve you well as a data scientist!

<br/><br/>
<hr style="border: 5px solid #8a8c8c;" />
<hr style="border: 1px solid #ffcd00;" />

## Congratulations! You have finished Homework 3! ##

### Submission Instructions

Below, you will see a cell. Running this cell will automatically generate a zip file with your autograded answers. Once you submit this file to the HW 3 Coding assignment on Gradescope, Gradescope will automatically submit a PDF file with your written answers to the HW 3 Written assignment. If you run into any issues when running this cell, feel free to check the [Debugging Guide](https://mtu.instructure.com/courses/1527249/pages/debugging-guide).

**Important**: Please check that your written responses were generated and submitted correctly to the HW 3 Written Assignment.

**You are responsible for ensuring your submission follows our requirements and that the PDF for HW 3 written answers was generated/submitted correctly. We will not be granting regrade requests nor extensions to submissions that don't follow instructions.** If you encounter any difficulties with submission, please don't hesitate to reach out to staff prior to the deadline. 

### Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()